In [1]:
import os
import time
import base64
import json
import requests
import sys
import os
import time
import base64
import json
import requests
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from function1 import send_text, click_element, set_date_field
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.firefox import GeckoDriverManager  # Added this import

In [2]:
firefox_options = Options()
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=firefox_options)
driver.get("https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html")



====== WebDriver manager ======
Current firefox version is 139.0
Get LATEST geckodriver version for 139.0 firefox
Driver [C:\Users\Admin\.wdm\drivers\geckodriver\win64\v0.36.0\geckodriver.exe] found in cache


In [3]:
with open("config_data.json", "r") as f:
    config_data = json.load(f)

In [4]:
        # Final form section - add error handling for elements that may not be present
time.sleep(1)
try:
            #*DIN/DPIN/PAN of designated partner
            send_text(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-guidetextbox_1527295062___widget', keys="00000001")
except Exception as e:
            print(f"DIN/DPIN/PAN field not found: {str(e)}")

In [5]:
# Declaration and certification by professional
# Enter Name
send_text(driver, css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guidetextbox___widget', keys=config_data['form_data']['fields']['Enter Name'])
print("success")

success


In [6]:
# Son/Daughter
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guideradiobutton__-1_widget')
print("success")

success


In [7]:
# Enter Father's Name
send_text(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guidetextbox_1141998439___widget', keys=config_data['form_data']['fields']["Enter Father's Name"])
print("success")

success


In [8]:
# do state that I am
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guideradiobutton_2109271305__-3_widget')
print("success")

success


In [9]:


# partnership and my membership number
send_text(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guidetextbox_1809173783___widget', keys=config_data['form_data']['fields']['partnership and my membership number'])
print("success")

success


In [10]:

# *Whether associate or fellow
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel-panel_1137005940_cop-guideradiobutton_598616795__-1_widget')
print("success")

success


In [11]:

# SAVE         
click_element(driver,css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel_copy_copy_copy-mca_button_copy___widget')
print("success")

success


In [12]:

click_element(driver, css_selector='#guideContainer-rootPanel-modal_container_copy-panel_86338280-panel-mca_button___widget')
print("success")

success


In [13]:

click_element(driver, css_selector='#guideContainer-rootPanel-panel-panel_1696210624-panel_1548670294-panel_copy_copy_copy-mca_button___widget')
print("success")

success


In [ ]:
# Proceed to Form 9
# click_element(driver, css_selector='#guideContainer-rootPanel-panel-panel_1448122692-panel_copy_copy_copy-mca_button___widget')
# print("success")


# Wait for the visible version of the button
wait = WebDriverWait(driver, 10)
visible_button = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, '#guideContainer-rootPanel-panel-panel_1448122692-panel_copy_copy_copy-mca_button___widget')
        ))

# Optional: confirm it's not inside a hidden div
hidden_parent = driver.find_element(By.ID, 'im_guideContainer-rootPanel-panel-panel_1448122692-panel_copy_copy_copy__')
if hidden_parent.is_displayed():
            print("[✗] Warning: Hidden version of button is visible. Wrong state.")
else:
            # Scroll & click the correct visible button
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", visible_button)
            ActionChains(driver).move_to_element(visible_button).click().perform()
            print("[✓] Proceed to Form 9 button clicked.")

[✓] Proceed to Form 9 button clicked.
